<a href="https://colab.research.google.com/github/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7/blob/main/DiffSinger_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _**[DiffSinger](https://github.com/openvpi/DiffSinger)**_
_Singing Voice Synthesis via Shallow Diffusion Mechanism (SVS & TTS)_

____

Note:
- This notebook will get update semi-frequently based from the feedback or response from users

\
____
\
#### **This notebook is an edited copy of Kei's Diffsinger [colab notebook](https://colab.research.google.com/drive/1kUg9dz8PPH92NfnLZwgq0_9B9an39t1J?usp=sharing)**
####**This notebook is maintained by MLo7**
\
___

```Expand this cell for more details```

This notebook converts your data (.lab & .wav) to compatible format via [nnsvs-db-converter](https://github.com/UtaUtaUtau/nnsvs-db-converter)

~~Extracted data will also make .ds files when estimate_midi is enabled, which might not be accurate by default. So you might want to edit it using [SlurCutter](https://github.com/openvpi/MakeDiffSinger/releases) for a refined data for your pitch model~~ (This notebook currently does not support pre-made files)

Preferred zip file format (to avoid difficulties):

<pre>
your_zip.zip:
    |
    |
    data1.wav
    data1.lab
    |
    |
    data2.wav
    data3.lab
    |
    |
    data3.wav
    data3.lab
    |
    ...
</pre>
This will be updated once multisinger training is added

_**Credits:** _

  - [openvpi](https://openvpi.github.io/) for DiffSinger fork and more

  - [UtaUtaUtau](https://utautautau.neocities.org/) for nnsvs-db-converter

  - [Kei](https://pronouns.page/@kei.wendt06) for the original notebook

  - [MLo7](https://github.com/MLo7Ghinsan) for the notebook edit

# **Setup**

In [1]:
#@markdown Select this if you don't like seeing warnings throughout your training since most of the time the warnings are nothing to worry about

#@markdown ****WARNING**** this will also hides the error message
no_warn = False # @param {type:"boolean"}

#@markdown <font size="-1.5"> you can always come back and enable or disable this cell without re-running the installation

In [ ]:
#@title # Mount Google Drive and Setup

from IPython.display import clear_output
from IPython.display import Audio, display, HTML
import os
from google.colab import drive
drive.mount("/content/drive")

if not os.path.exists("/content/play_sound"):
    os.makedirs("/content/play_sound")
%cd /content/play_sound
!wget -O setup_complete.wav https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/audio/setup_complete.wav
%cd /content
!rm -rf /content/sample_data
!apt-get install aria2
clear_output()

!git clone https://github.com/UtaUtaUtau/nnsvs-db-converter
!git clone https://github.com/openvpi/MakeDiffSinger
!git clone https://github.com/openvpi/DiffSinger.git
clear_output()
!pip install torch==1.13.0 torchvision==0.14.0 torchaudio==0.13.0
clear_output()
!pip install -r /content/DiffSinger/requirements.txt
clear_output()
!pip install onnx onnxsim #onnx==1.12.0 onnxsim==0.4.10
clear_output()
!aria2c https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip
!aria2c https://github.com/openvpi/DiffSinger/releases/download/v2.1.0/rmvpe.zip
!unzip -q /content/nsf_hifigan_20221211.zip -d /content/DiffSinger/checkpoints
!unzip -q /content/rmvpe.zip -d /content/DiffSinger/checkpoints
!unzip -q /content/rmvpe.zip -d /content/MakeDiffSinger/variance-temp-solution/assets
!rm /content/nsf_hifigan_20221211.zip
!rm /content/rmvpe.zip
clear_output()
!pip install --upgrade tensorboard
clear_output()
!pip install protobuf #protobuf==3.20
clear_output()
!pip install onnxruntime
clear_output()
#shit tons of clear output cus i dont wanna see anything <3

print("setup complete!")
print("|")
print("|")
print("|")

chika_dance = '<img src="https://cdn.discordapp.com/attachments/816517150175920138/1090112497446563950/icegif-2013.gif"/>'
display(HTML(chika_dance))

with open("/content/play_sound/setup_complete.wav", "rb") as f:
    setup_complete_sound = f.read()
Audio(data=setup_complete_sound, autoplay=True)

# **Preprocess data for training**

In [ ]:
#@title #Extract Data
%cd /content
#@markdown this cell will create a folder name [raw_data] in the root folder and extract your data into it

#@markdown you can go and check to make sure if it actually unzips your data or not

#@markdown <font size="-1.5"> The path to your zip file containing .lab and .wav files

data_zip_path = "" #@param {type:"string"}

#@markdown <font size="-1.5"> Use this if your data is not in diffsinger's preferred format (data are under 30 seconds | have "AP" label in your lab)

default_converter_setting = True # @param {type:"boolean"}

#@markdown ___

#@markdown this lower section is for variance training

#@markdown <font size="-1.5"> Use this if you don't have .cvs that is for variance dataset (skippable if you are doing acoustic)

estimate_midi = False # @param {type:"boolean"}

#@markdown <font size="-1.5"> Pitch extractor algorithm - for converting your data to DB-ready format (rmvpe is more accurate but has range limit)

f0_ext = "parselmouth" # @param ["parselmouth", "rmvpe"]

all_shits = "/content/raw_data"

import os
import zipfile
if not os.path.exists(all_shits):
  os.makedirs(all_shits)

#changed from !unzip cus it kinda stopped working for some reason
with zipfile.ZipFile(data_zip_path, "r") as zip_ref:
    file_names = zip_ref.namelist()
wav_files = [file_name for file_name in file_names if file_name.endswith('.wav')]
with zipfile.ZipFile(data_zip_path, "r") as zip_ref:
    zip_ref.extractall(path=all_shits, members=wav_files)
lab_files = [file_name for file_name in file_names if file_name.endswith('.lab')]
with zipfile.ZipFile(data_zip_path, "r") as zip_ref:
    zip_ref.extractall(path=all_shits, members=lab_files)

if default_converter_setting:
    !python /content/nnsvs-db-converter/db_converter.py -s 2 {all_shits} 2> /dev/null # -s 2 cus pix tutorial said so <3
    clear_output()
else:
    !python /content/nnsvs-db-converter/db_converter.py -s 50 -S 20 -l 35 {all_shits} 2> /dev/null # old param set by me lmao uwu
    clear_output()

#funny auto dict generator lmao
lab_folder_path = "/content/raw_data"
out = "/content/DiffSinger/dictionaries/custom_dict.txt"
phonemes = set()
for root, dirs, files in os.walk(lab_folder_path):
    for file in files:
        if file.endswith(".lab"):
            fpath = os.path.join(root, file)
            with open(fpath, "r") as lab_file:
                for line in lab_file:
                    line = line.strip()
                    if line:
                        phoneme = line.split()[2]
                        if phoneme != "pau" and phoneme != "AP" and phoneme != "SP":
                            phonemes.add(phoneme + "	" + phoneme)
phonemes_mess = phonemes
with open(out, "w") as f:
    for phoneme in sorted(phonemes_mess):
        f.write(phoneme + "\n")

if estimate_midi:
    #prepare stuff for uhhhh variance even though it wouldnt be that good out of the box for pitch training

    dict_path = out

    vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
    vowel_data = []
    consonant_data = []

    with open(dict_path, "r") as f:
        for line in f:
            phoneme, _ = line.strip().split("\t")
            if phoneme[0] in vowel_types:
                vowel_data.append(phoneme)
            else:
                consonant_data.append(phoneme)

    vowel_data.sort()
    consonant_data.sort()
    directory = os.path.dirname(dict_path)
    #vowels.txt
    vowel_txt_path = os.path.join(directory, "vowels.txt")
    with open(vowel_txt_path, "w") as f:
        f.write(" ".join(vowel_data))
    #consonants.txt
    consonant_txt_path = os.path.join(directory, "consonants.txt")
    with open(consonant_txt_path, "w") as f:
        f.write(" ".join(consonant_data))

    # idk i just feel like 800 is a lil low for some people
    new_f0_max = 1760
    og_script = "/content/MakeDiffSinger/variance-temp-solution/get_pitch.py"
    with open(og_script, 'r') as file:
        mate = file.read()
    up_f0_val = mate.replace("f0_max = 800", f"f0_max = {new_f0_max}")
    with open(og_script, 'w') as file:
        file.write(up_f0_val)

    if no_warn:
        !python /content/MakeDiffSinger/variance-temp-solution/add_ph_num.py /content/raw_data/diffsinger_db/transcriptions.csv --vowels /content/DiffSinger/dictionaries/vowels.txt --consonants /content/DiffSinger/dictionaries/consonants.txt 2> /dev/null
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/estimate_midi.py /content/raw_data/diffsinger_db/transcriptions.csv /content/raw_data/diffsinger_db/wavs --pe {f0_ext} 2> /dev/null
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/convert_ds.py csv2ds /content/raw_data/diffsinger_db/transcriptions.csv /content/raw_data/diffsinger_db/wavs 2> /dev/null
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/correct_cents.py ds /content/raw_data/diffsinger_db/wavs 2> /dev/null
        clear_output()
        !rm -rf {all_shits}/diffsinger_db/transcriptions.csv
        !rm /content/raw_data/diffsinger_db/curves.json
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/convert_ds.py ds2csv /content/raw_data/diffsinger_db/wavs /content/raw_data/diffsinger_db/transcriptions.csv 2> /dev/null
        clear_output()
    else:
        !python /content/MakeDiffSinger/variance-temp-solution/add_ph_num.py /content/raw_data/diffsinger_db/transcriptions.csv --vowels /content/DiffSinger/dictionaries/vowels.txt --consonants /content/DiffSinger/dictionaries/consonants.txt
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/estimate_midi.py /content/raw_data/diffsinger_db/transcriptions.csv /content/raw_data/diffsinger_db/wavs --pe {f0_ext}
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/convert_ds.py csv2ds /content/raw_data/diffsinger_db/transcriptions.csv /content/raw_data/diffsinger_db/wavs
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/correct_cents.py ds /content/raw_data/diffsinger_db/wavs
        clear_output()
        !rm /content/raw_data/diffsinger_db/transcriptions.csv
        !rm /content/raw_data/diffsinger_db/curves.json
        clear_output()
        !python /content/MakeDiffSinger/variance-temp-solution/convert_ds.py ds2csv /content/raw_data/diffsinger_db/wavs /content/raw_data/diffsinger_db/transcriptions.csv
        clear_output()
else:
    pass
print("extraction complete!")
print("|")
print("|")
print("|")
print("I'm also nice enough to convert your data and also write your dict.txt lmao. You are welcome :)")

In [ ]:
#@title #Edit Config

%cd /content
clear_output()
#@markdown <font size="-1.5"> The training type you want to do
config_type = "acoustic" # @param ["acoustic", "variance"]
config_cap = config_type.upper()
#@markdown <font size="-1.5"> The name of your speaker
spk_name = "" #@param{type:"string"}

#@markdown <font size="-1.5"> Path to where you want to save your binary data for later use
binary_save_dir = "" #@param{type:"string"}

#@markdown <font size="-1.5"> Pitch extractor algorithm

f0_ext = "parselmouth" # @param ["parselmouth", "rmvpe"]
if f0_ext == "rmvpe":
    pe_ckpt_pth = "checkpoints/rmvpe/model.pt"
else:
    pe_ckpt_pth = None

#@markdown <font size="-1.5"> Select this is you want to use data augmentation (default pitch shift and time stretch values)
data_aug = False #@param {type:"boolean"}

#@markdown <font size="-1.5"> Step interval of when your model will be validate and save
save_interval = 2000 #@param {type:"slider", min:100, max:10000, step:100}

#@markdown <font size="-1.5"> Your model save path
save_dir = "" #@param{type:"string"}

import os
import yaml
import random #for the random test files lmaoz

wav_files = [f for f in os.listdir("/content/raw_data/diffsinger_db/wavs") if f.endswith(".wav")]
random.shuffle(wav_files)
random_ass_wavs = wav_files[:3]
random_ass_test_files = [os.path.splitext(file)[0] for file in random_ass_wavs]

if config_type == "acoustic":
    with open("/content/DiffSinger/configs/acoustic.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = [spk_name]
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = "/content/raw_data/diffsinger_db"
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["augmentation_args"]["random_pitch_shifting"]["enabled"] = data_aug
    bitch_ass_config["augmentation_args"]["random_time_stretching"]["enabled"] = data_aug
    bitch_ass_config["use_key_shift_embed"] = data_aug
    bitch_ass_config["use_speed_embed"] = data_aug
    bitch_ass_config["max_batch_size"] = 9 #ive never tried reaching the limit so ill trust kei's setting for this
    bitch_ass_config["val_check_interval"] = save_interval
    bitch_ass_config["pe"] = f0_ext
    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth
    with open("/content/DiffSinger/configs/acoustic.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)
else:
    with open("/content/DiffSinger/configs/variance.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = [spk_name]
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = "/content/raw_data/diffsinger_db"
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["max_batch_size"] = 9 #ive never tried reaching the limit so ill trust kei's setting for this
    bitch_ass_config["val_check_interval"] = save_interval
    bitch_ass_config["pe"] = f0_ext # i think variance uses it for pitch ref as ground-truth for pitch training soooo
    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth #same goes to this one
    with open("/content/DiffSinger/configs/variance.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)

os.makedirs(save_dir, exist_ok=True)
search_text = "        args_work_dir = os.path.join("
replacement = f"        args_work_dir = '{save_dir}'"
with open("/content/DiffSinger/utils/hparams.py", "r") as file:
    lines = file.readlines()
for i, line in enumerate(lines):
    if search_text in line:
        lines[i] = replacement + "\n"
        break
with open("/content/DiffSinger/utils/hparams.py", "w") as file:
        file.writelines(lines)
#incase if anyone wanna change it lmao
search_text_alt = "        args_work_dir = '"
replacement_alt = f"        args_work_dir = '{save_dir}'"
with open("/content/DiffSinger/utils/hparams.py", "r") as file:
    lines = file.readlines()
for i, line in enumerate(lines):
    if search_text_alt in line:
        lines[i] = replacement_alt + "\n"
        break
with open("/content/DiffSinger/utils/hparams.py", "w") as file:
        file.writelines(lines)

relative_p = "            relative_path = filepath.relative_to(Path('.').resolve())"
relative_change = "            relative_path = filepath.relative_to(Path('/content').resolve())"
with open("/content/DiffSinger/utils/training_utils.py", "r") as file:
    lines = file.readlines()
for i, line in enumerate(lines):
    if relative_p in line:
        lines[i] = relative_change + "\n"
        break
with open("/content/DiffSinger/utils/training_utils.py", "w") as file:
        file.writelines(lines)
relative_p_2 = "        relative_path = filepath.relative_to(Path('.').resolve())"
relative_change_2 = "        relative_path = filepath.relative_to(Path('/content').resolve())"
with open("/content/DiffSinger/utils/training_utils.py", "r") as file:
    lines_2 = file.readlines()
for i, line in enumerate(lines):
    if relative_p_2 in line:
        lines_2[i] = relative_change_2 + "\n"
        break
with open("/content/DiffSinger/utils/training_utils.py", "w") as file:
        file.writelines(lines_2)

if not estimate_midi:
    !python /content/DiffSinger/scripts/migrate.py txt /content/raw_data/diffsinger_db/transcriptions.txt 2> /dev/null
else:
  pass

print("config updated! see below for config's information")
print("|")
print("|")
print("|")
print(f"+++---{config_cap} TRAINING---+++")
print("|")
print("|")
print("|")
print("+++---user's settings---+++")
print("\n")
print(f"speaker name: {spk_name}")
print("\n")
print(f"data augmentation: {data_aug}")
print("\n")
print(f"pitch extractor: {f0_ext}")
print("\n")
print(f"binary data save directory: {binary_save_dir}")
print("\n")
print(f"your model will be saved every: {save_interval} steps")
print("\n")
print(f"your model will be saved to: {save_dir}")
print("\n")
print("==========================================================================================")
print("\n")
print("+++---other auto-defined settings---+++")
print("\n")
print(f"test files (auto selected 3 files): {random_ass_test_files}")
print("\n")
print("dictionary (auto generated): custom_dict.txt")
print("\n")
print("max_sentences: 9")
print("\n")
print("==========================================================================================")
print("\n")
print("if you don't like or disagree with any of these options,")
print(f"you can go and edit the config at [/content/DiffSinger/configs/{config_type}.yaml]")


In [ ]:
#@markdown # Preprocess data
import os

# idk i just feel like 800 is a lil low for some people part 2
new_f0_max = 1600
og_script = "/content/DiffSinger/utils/binarizer_utils.py"
with open(og_script, 'r') as file:
    mate = file.read()
up_f0_val = mate.replace("f0_max = 800", f"f0_max = {new_f0_max}")
with open(og_script, 'w') as file:
    file.write(up_f0_val)

training_config = f"/content/DiffSinger/configs/{config_type}.yaml"

%cd /content/DiffSinger
os.environ['PYTHONPATH']='.'
if no_warn:
    !CUDA_VISIBLE_DEVICES=0 python /content/DiffSinger/scripts/binarize.py --config {training_config} 2> /dev/null
else:
    !CUDA_VISIBLE_DEVICES=0 python /content/DiffSinger/scripts/binarize.py --config {training_config}

# **Training**

In [ ]:
#@markdown # Tensorboard

#@markdown For monitoring training progress. Enter the directory to your model save location (save_dir)

#@markdown <font size="-1.5"> if you are continuing from latest checkpoint, this would be the directory of a folder that you saved your model, it should have [lightning_logs] folder in it

logs = "" #@param{type:"string"}
%reload_ext tensorboard
%tensorboard --logdir {logs}/lightning_logs

In [ ]:
#@markdown #Train your model
%cd /content/DiffSinger
#@markdown Enter the name of your singer.
name = "" #@param{type:'string'}

#@markdown ___

#@markdown ###**Only use this lower section if you want to resume training**
resume_training = False #@param {type:"boolean"}

#@markdown <font size="-1.5"> path to the config you got from training
re_config_path = "" #@param {type:"string"}

#@markdown <font size="-1.5"> path to the resume model's **FOLDER** (should mostlikely be the path you put above minus [ /config.yaml ])

model_dir = "" #@param {type:"string"}

if resume_training:
    config_path = re_config_path
    search_text = "        args_work_dir = os.path.join("
    replacement = f"        args_work_dir = '{model_dir}'"
    with open("/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text in line:
            lines[i] = replacement + "\n"
            break
    with open("/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)
    #incase if anyone wanna change it lmao
    search_text_alt = "        args_work_dir = '"
    replacement_alt = f"        args_work_dir = '{model_dir}'"
    with open("/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text_alt in line:
            lines[i] = replacement_alt + "\n"
            break
    with open("/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)

    relative_p = "            relative_path = filepath.relative_to(Path('.').resolve())"
    relative_change = "            relative_path = filepath.relative_to(Path('/content').resolve())"
    with open("/content/DiffSinger/utils/training_utils.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if relative_p in line:
            lines[i] = relative_change + "\n"
            break
    with open("/content/DiffSinger/utils/training_utils.py", "w") as file:
            file.writelines(lines)
    relative_p_2 = "        relative_path = filepath.relative_to(Path('.').resolve())"
    relative_change_2 = "        relative_path = filepath.relative_to(Path('/content').resolve())"
    with open("/content/DiffSinger/utils/training_utils.py", "r") as file:
        lines_2 = file.readlines()
    for i, line in enumerate(lines):
        if relative_p_2 in line:
            lines_2[i] = relative_change_2 + "\n"
            break
    with open("/content/DiffSinger/utils/training_utils.py", "w") as file:
            file.writelines(lines_2)
    !cp {model_dir}/dictionary.txt /content/DiffSinger/dictionaries/custom_dict.txt

else:
    config_path = training_config

if no_warn:
    !CUDA_VISIBLE_DEVICES=0 python /content/DiffSinger/scripts/train.py --config {config_path} --exp_name {name} --reset 2> /dev/null
else:
    !CUDA_VISIBLE_DEVICES=0 python /content/DiffSinger/scripts/train.py --config {config_path} --exp_name {name} --reset

# **Convert model to ONNX format**

In [ ]:
#@markdown # Convert to ONNX for OpenUtau
%cd /content

#@markdown <font size="-1.5"> the type of the model you want to convert
model_type = "acoustic" # @param ["acoustic", "variance"]

#@markdown <font size="-1.5"> path to your checkpoint's **FOLDER** (NOT the model itself) or path to your save_dir
checkpoints_path = "" #@param{type:"string'}
folder_name = os.path.basename(checkpoints_path)

#@markdown <font size="-1.5"> path to where you want to save your converted model and it's file
exp_folder = "" #@param{type:"string"}

!cp {checkpoints_path} -r /content/DiffSinger/checkpoints
if no_warn:
    !python /content/DiffSinger/scripts/export.py {model_type} --exp {folder_name} --out {exp_folder} 2> /dev/null
else:
    !python /content/DiffSinger/scripts/export.py {model_type} --exp {folder_name} --out {exp_folder}

# **Miscellaneous**

In [ ]:
import os

#@title Generate enunux.yaml

#@markdown <font size="-2.5"> path to your dictionary.txt

dict_path = "" #@param{type:"string'}
enunux = "enunux.yaml"
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
enunux_data = []
vowel_data = []
stop_data = []
with open(dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        phoneme_type = "vowel" if phoneme[0] in vowel_types else "stop"
        entry = f"- {{\"symbol\": \"{phoneme}\", \"type\": \"{phoneme_type}\"}}"
        if phoneme_type == "vowel":
            vowel_data.append(entry)
        else:
            stop_data.append(entry)
vowel_data.sort()
stop_data.sort()
enunux_data.extend(["# Vowel type symbols:", *vowel_data, "", "# Stop type symbols:", *stop_data])
directory = os.path.dirname(dict_path)
enunux_path = os.path.join(directory, enunux)
with open(enunux_path, "w") as f:
    f.write("symbols:\n")
    f.write("\n".join(enunux_data))
    f.write("\n")

In [ ]:
#@title Make OpenUtau compatible voicebank
#@markdown not working yet lmao COMING NEXT UPDATE THO I PROMISE

# Last Section Note
Wow you made it to the very bottom.... Why though lmao hahahahhshahhasdksajidhasjl

Anyways, now that you are here i guess ill tell you my plan/todo list for this notebook \
(feel free to suggest anything via [discord](https://discord.com/invite/wwbu2JUMjj) my user display name is MLo7 and my user name is ghin_mlo7)

todo list:
- add support for premade/refined data
- add multi-singer training
- add OpenUtau voicebank builder
- add link to vocoder training notebook (yet to be ready) or add a vocoder training section

If you want to add anything to this list then again, just ping or message me lmao